In [11]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings() 
# Load vectorstores
pdf_vectorstore = FAISS.load_local("faiss_index_rules_egypt", embeddings,allow_dangerous_deserialization=True)  # assuming you saved it similarly
csv_vectorstore = FAISS.load_local("faiss_index_with_csv", embeddings,allow_dangerous_deserialization=True)  # assuming you saved it similarly
# Combine vectorstores
#combined_vectorstore = FAISS.combine([pdf_vectorstore, csv_vectorstore])
## Save the combined vectorstore
#combined_vectorstore.save_local("faiss_index_combined", embeddings)

In [12]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.llms import OpenAI
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)
# Load embeddings
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# Load vectorstores
pdf_vectorstore = FAISS.load_local("faiss_index_rules_egypt", embeddings, allow_dangerous_deserialization=True)
csv_vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

# Define prompts
Rules_template = """You are a knowledgeable lawyer specializing in tourism laws in Egypt.
Use the following context to help answer the question:
{context}

Question:
{query}"""

restaurant_template = """You are a highly experienced tour guide specializing in restaurants.
Use the following context to help answer the question:
{context}

Question:
{query}"""

# Map of prompts and retrievers
template_map = {
    "rules": PromptTemplate.from_template(Rules_template),
    "restaurant": PromptTemplate.from_template(restaurant_template),
}

retrievers = {
    "rules": pdf_vectorstore.as_retriever(search_kwargs={"k": 4}),
    "restaurant": csv_vectorstore.as_retriever(search_kwargs={"k": 4}),
}

# 🧠 Use clear task descriptions instead of full templates for routing
routing_labels = ["tourism laws in Egypt", "restaurant recommendations"]
routing_embeddings = embeddings.embed_documents(routing_labels)

# LLM setup
llm = OpenAI(temperature=0)

# Routing function
def prompt_router(input):
    query = input.get("query", "")
    if not query or not isinstance(query, str):
        raise ValueError("Query must be a valid string")

    query_embedding = embeddings.embed_query(query)
    similarity = cosine_similarity([query_embedding], routing_embeddings)[0]
    most_similar_idx = similarity.argmax()

    source_key = ["rules", "restaurant"][most_similar_idx]
    print(f"🔀 Routed to: {source_key} (Score: {similarity[most_similar_idx]:.2f})")

    retriever = retrievers[source_key]
    context_docs = retriever.get_relevant_documents(query)
    context = "\n".join(doc.page_content for doc in context_docs)

    return {
        "query": query,
        "context": context,
        "prompt": template_map[source_key]
    }

# Final chain
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | RunnableLambda(lambda input: input["prompt"].format(query=input["query"], context=input["context"]))
    | llm
    | StrOutputParser()
)


In [13]:
# Test the chain
response = chain.invoke("suggest agood restaurant in Cairo for a family with kids")
print(response)

🔀 Routed to: restaurant (Score: 0.85)


As a highly experienced tour guide specializing in restaurants, I would recommend Baba El Masry restaurant for families with kids in Cairo. It has a high number of reviews and is located in a central area. The menu offers a variety of options suitable for children and adults, and the atmosphere is family-friendly. Additionally, the prices are reasonable and the food is delicious. I am confident that your family will have a great dining experience at Baba El Masry.
